In [1]:
from copy import deepcopy
from os import environ
from pathlib import Path

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, make_scorer
from requests import get, Response
from hashlib import sha256
from tqdm.notebook import tqdm
from zipfile import ZipFile
from IPython.display import display, Markdown
import pandas as pd
import missingno as msno
import seaborn as sns
import numpy as np
from IPython.display import clear_output
from lightgbm import LGBMClassifier
from lightgbm import LGBMClassifier

import re
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
# Added imports for the new transformer
from scipy import stats
from random import sample

from lightgbm import LGBMClassifier
from deliverables.utils.image_inverter import save
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, cross_validate
from mlflow import set_tracking_uri
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Partie 0 - Constantes, imports et outils

In [2]:
_cache_folder = Path('~/.cache/gn_p7').expanduser()
_cache_folder.mkdir(parents=True, exist_ok=True)

_ds_url = 'https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Impl%C3%A9menter+un+mod%C3%A8le+de+scoring/Projet+Mise+en+prod+-+home-credit-default-risk.zip'

graph_folder: Path = Path("./graphs")


def save_figure(figure: plt.Figure, folder: str, figure_name: str) -> None:
    folder = graph_folder / folder
    folder.mkdir(parents=True, exist_ok=True)
    save(figure, folder / f'{figure_name}.png', close=True)


def download(url: str) -> Path:
    url_id: str = sha256(url.encode('utf-8')).hexdigest()
    local_path: Path = _cache_folder / url_id
    local_path.parent.mkdir(parents=True, exist_ok=True)
    if not local_path.exists():
        tmp_path: Path = _cache_folder / (url_id + '.tmp')
        res: Response = get(url, stream=True)
        with tmp_path.open('wb') as f, tqdm(
                total=int(res.headers.get('content-length')),
                desc=f'Downloading {url}',
                unit_scale=True) as q:
            for chunk in res.iter_content(chunk_size=8192):
                q.update(len(chunk))
                f.write(chunk)
        tmp_path.replace(local_path)
    return local_path


def download_zip_archive(url: str) -> Path:
    """Download a zip archive, extract it then return the folder containing its content"""
    archive_path: Path = download(url)
    archive_folder: Path = Path(archive_path.as_posix() + '.dir')

    if not archive_folder.exists():
        print(f'Extracting archive {url}...', flush=True)
        archive_temp: Path = Path(archive_path.as_posix() + '.tmp')
        archive_temp.mkdir(parents=True, exist_ok=True)
        archive: ZipFile = ZipFile(archive_path)
        archive.extractall(path=archive_temp)
        archive_temp.replace(archive_folder)
        print(f'Extracting archive {url}...done', flush=True)

    return archive_folder


datasets: dict[str, pd.DataFrame] = {}


def get_dataset(name: str) -> pd.DataFrame:
    folder = download_zip_archive(_ds_url)
    if not name.endswith('.csv'):
        name = f'{name}.csv'
    try:
        return datasets[name]
    except KeyError:
        try:
            _df = pd.read_csv(folder / name)
        except FileNotFoundError:
            display(Markdown(f'# ERROR: Dataset {name!r} not found, available datasets are:\n' + '\n'.join(
                f'- {p.name}' for p in sorted(folder.iterdir(), key=(lambda x: x.name.lower())))))
            raise KeyError(name) from None
        else:
            datasets[name] = _df
            return _df.copy()


# Partie 1 - EDA

## Partie 1.0 - Chargement des données

In [3]:
train, test = map(get_dataset, ('application_train', 'application_test'))

In [4]:
train_df = get_dataset('application_train')
test_df = get_dataset('application_test')

# Séparation des features et de la cible
X = train_df.drop('TARGET', axis=1)
y = train_df['TARGET']

print("Données d'entraînement chargées :", X.shape)
print("Cible chargée :", y.shape)

Données d'entraînement chargées : (307511, 121)
Cible chargée : (307511,)


In [5]:
train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
test

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


## Partie 1.1 - Analyse de la cible

La cible est présente dans le dataset d'entrainement mais pas dans le dataset de test, pour éviter les fuites de données.

In [7]:
save_figure(train.TARGET.value_counts().plot.pie(
    title='Répartition des cibles (0=paiement complet, 1=retards de paiement)'
).figure, '1_model', '0_target')

## Partie 1.3 - Analyse des features (hors cible)

In [8]:
# Function to calculate missing values by column# Funct
def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values'})

    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

    # Print some summary information
    print("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
                                                              "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")

    # Return the dataframe with missing information
    return mis_val_table_ren_columns


def missing_stats():
    for fold in ('train', 'test'):
        msno.matrix(get_dataset('application_' + fold + '.csv'), fontsize=12)
        plt.title(f'Missing Values Count ({fold.title()}ing fold)', fontsize=16)
        save_figure(plt.gcf(), '1_model', '1_missing_' + fold.title())

    for fold in ('train', 'test'):
        df = get_dataset('application_' + fold + '.csv')
        msno.matrix(df[list(sorted(df.columns, key=(lambda col: int(df[col].notna().sum()))))], fontsize=12)
        plt.title(f'Missing Values Count ({fold.title()}ing fold)', fontsize=16)
        save_figure(plt.gcf(), '1_model', '2_sorted_missing_' + fold.title())

    for fold in ('train', 'test'):
        missing_test_values = missing_values_table(get_dataset('application_' + fold + '.csv'))

        # TODO: Set the plot style for dark mode when exporting to png
        plt.figure(figsize=(16, 12))  # There are a lot of columns
        sns.barplot(x=missing_test_values['% of Total Values'], y=missing_test_values.index)
        plt.title(f'Percentage of Missing Values by Feature ({fold.title()}ing fold)', fontsize=16)
        plt.xlabel('% of Total Values', fontsize=12)
        plt.ylabel('Features', fontsize=12)

        # Add percentage text on the bars
        for index, value in enumerate(missing_test_values['% of Total Values']):
            plt.text(value, index, f' {value}%', va='center')

        plt.xlim(0, 110)  # Set x-limit to give space for text
        plt.tight_layout()
        save_figure(plt.gcf(), '1_model', '3_graph_missing' + fold.title())


missing_stats()

Your selected dataframe has 122 columns.
There are 67 columns that have missing values.
Your selected dataframe has 121 columns.
There are 64 columns that have missing values.


Nous pouvons voir qu'à peu près la moitié des colonnes manquent au moins une valeur, et que le reste est défini à environ 45-75%
Si nous nous intéressons

In [9]:
assert not len(train.columns[
                   (train.dtypes != 'int64') &
                   (train.dtypes != 'float64') &
                   (train.dtypes != 'object')]), 'Plus de types de colonnes sont présentes'
display(Markdown('Il existe trois types de données en entrée, int64 et float64, numériques, et object, catégorielles'))
display(
    Markdown('Il arrive parfois que des données numériques soient accidentellement catégorisées en "object" si elles'
             ' contiennent des valeurs non numérique, ce n\'est pas le cas ici'))

Il existe trois types de données en entrée, int64 et float64, numériques, et object, catégorielles

Il arrive parfois que des données numériques soient accidentellement catégorisées en "object" si elles contiennent des valeurs non numérique, ce n'est pas le cas ici

## Partie 2 - Définition du Score Métier

Nous somme face à deux déséquilibres:
- Un déséquilibre des coûts (pb métier) : un faux négatif coûtant dix fois plus cher qu'un faux positif, il faut en tenir compte pour minimiser plus fortement les faux négatifs.
    - Nous devons créer un `scorer` pour Scikit-Learn qui minimise le coût `coût = 10 * FN + 1 * FP`
    - Il doit aussi trouver le seuil de décision optimal, car le seuil par défaut de 0.5 n'est probablement pas le meilleur d'un point de vue métier.
- Un déséquilibre des classes (pb données) : 92% des cibles étant négatives, un risque serai accru d'avoir un modèle prédisant trops 0 (prédire 0 tout le temps nous donnera 92% de précision.
    - Cela sera traité en utilisant comme paramètre `class_weight='balanced'`
    - TODO: Don't forget to include it every time I introduce a new model

In [10]:
def find_optimal_threshold(y_true: np.ndarray[..., ...], y_pred_proba: np.ndarray[..., ...]):
    """Trouve le seuil qui minimise le coût métier."""
    thresholds = np.linspace(0.01, 0.99, 100)
    costs = []
    for t in thresholds:
        y_pred = (y_pred_proba[:, 1] >= t).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
        costs.append(10 * fn + 1 * fp)

    optimal_t_index = np.argmin(costs)
    return thresholds[optimal_t_index], costs[optimal_t_index]


def business_cost_scorer_func(y_true: np.ndarray[..., ...], y_pred_proba: np.ndarray[..., ...]):
    """Calcule le coût métier minimal en trouvant le meilleur seuil."""
    _, min_cost = find_optimal_threshold(y_true, y_pred_proba)
    return min_cost


def optimal_threshold_scorer_func(y_true: np.ndarray[..., ...], y_pred_proba: np.ndarray[..., ...]):
    """Retourne le seuil optimal qui minimise le coût métier."""
    optimal_t, _ = find_optimal_threshold(y_true, y_pred_proba)
    return optimal_t


# Création des scorers pour Scikit-Learn
# On veut MINIMISER le coût, donc greater_is_better=False
business_scorer = make_scorer(business_cost_scorer_func, needs_proba=True, greater_is_better=False)

# Pour le seuil, c'est juste une information, donc pas de notion de 'meilleur'
threshold_scorer = make_scorer(optimal_threshold_scorer_func, needs_proba=True)

# On définit le dictionnaire de scoring qu'on utilisera dans la cross-validation
scoring = {
    'roc_auc': 'roc_auc',
    'business_cost': business_scorer,
    'optimal_threshold': threshold_scorer
}

print("Scorers métier créés avec succès.")

Scorers métier créés avec succès.


# Partie 3 - Définition d'un pipeline de prétraitement

L'avantage que le pipeline de prétraitement a est la robustesse contrer le data leakage, en effet les modèles d'apprentissages et de traitement seront entraîné sur les mêmes données, ce qui sera obligatoire pour pouvoir utiliser des techniques de K Fold en s'assurant que les folds soient indépendants les uns des autres.

Note: Nous allons ignorer la transformation BoxCox en premier lieu

Note relatives au déséquilibre de la cible:<br>
Il existe un déséquilibre flagrant entre 1 et 0, avec 1 n'étant présent que pour 8% des données d'entraînement.<br>
Pour y remédier, on a a notre disposition:
- oversampling, par exemple SMOT: Synthetic Minority OverSampling Technique
- undersampling
- cost sensitive learning: Ajuster le scoring pour pénaliser davantage les faux négatifs



TODO: Revoir s'il est nécessaire de tester les trois, ou si cost sensitive learning est suffisant


In [11]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from functools import wraps
from typing import Callable, Any


class OccupationPCA(BaseEstimator, TransformerMixin):
    """
    ORGANIZATION_TYPE et OCCUPATION_TYPE donnent 78 colonnes après OneHot, ce qui complexifie le dataset.
    En théorie cela nous donne 37 colonnes post pca
    Pour minimiser le nombre de features, nous allons procéder de la manière suivante:
    1. Application d'un One-Hot Encoding sur les colonnes 'ORGANIZATION_TYPE' et 'OCCUPATION_TYPE'.
    2. Application d'une ACP pour conserver 95% de la variance, en nommant composantes 'OT_0', 'OT_1', etc.
    5. Suppression des colonnes d'origine et ajout des nouvelles composantes au DataFrame.
    """

    def __init__(self, n_components=0.95):
        self.n_components = n_components  # Will either be a variance or number of components
        self.pca = PCA(n_components=self.n_components)  # This will be "fitted" with the estimator

        # To make sure the dataset holds the same shape, even if some values are missing from the testing dataset
        self.dummy_columns = []

    # TODO: Dummy NA?
    def fit(self, X, y=None):
        x = pd.get_dummies(X[['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']], dummy_na=True)
        self.dummy_columns = x.columns
        self.pca.fit(x)
        return self

    def transform(self, X):
        x = X.copy()
        x_org = pd.get_dummies(x[['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']], dummy_na=True)

        # Add, remove and move around columns to get the same shape as the training set
        x_org = x_org.reindex(columns=self.dummy_columns, fill_value=0)

        pca_result = self.pca.transform(x_org)  # Apply pre-trained PCA

        # Turning the results into a dataframe to include to x
        pca_df = pd.DataFrame(pca_result, columns=[f"OT_{i}" for i in range(pca_result.shape[1])], index=x.index)

        # Replacing old values with new pca values
        return pd.concat([x.drop(columns=['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']), pca_df], axis=1)



class CategoricalEncoder(BaseEstimator, TransformerMixin):

    def __init__(self, handle_unknown='ignore', sparse_output=False):
        self.handle_unknown = handle_unknown
        self.sparse_output = sparse_output
        self.ohe = OneHotEncoder(handle_unknown=self.handle_unknown,
                                 sparse_output=self.sparse_output)

    def fit(self, X, y=None):
        self.categorical_columns_ = X.select_dtypes(include=['object']).columns.tolist()
        if self.categorical_columns_:
            self.ohe.fit(X[self.categorical_columns_])
        return self

    def transform(self, X):
        X_transformed = X.copy()
        if self.categorical_columns_:
            encoded_data = self.ohe.transform(X[self.categorical_columns_])
            new_cols = self.ohe.get_feature_names_out(self.categorical_columns_)
            encoded_df = pd.DataFrame(encoded_data, columns=new_cols, index=X.index)
            X_transformed = X_transformed.drop(columns=self.categorical_columns_)
            X_transformed = pd.concat([X_transformed, encoded_df], axis=1)
        return X_transformed


def copy_and_return_x[T, U](callback: Callable[[T, U], None]) -> Callable[[T, U], T]:
    @wraps(callback)
    def wrapped(x: T, y: U = None) -> T:
        callback((x := x.copy()), y)
        return x

    return wrapped


@FunctionTransformer
@copy_and_return_x
def custom_preprocessor(X, y=None) -> None:
    # On commence par réparer l'erreur que nous avons vu à l'étape d'Analyse Exploratoire
    X['DAYS_EMPLOYED'] = X['DAYS_EMPLOYED'].replace({365243: np.nan})

    # On va ensuite supprimer la variable identifiant
    X.pop('SK_ID_CURR')  # TODO: Check for this

    # On va enfin traiter les valeurs catégorielles

    # TODO: Vaut-il mieux utiliser trois colonnes?
    X['CODE_GENDER'] = X['CODE_GENDER'].map({'F': 1, 'M': -1}).fillna(0)

    for flag_name in X.columns[X.columns.str.startswith('FLAG_')]:
        values = set(X[flag_name])
        if {0, 1} - values:
            continue
        if {"Y", "N"} - values:
            X[flag_name] = X[flag_name].map({'Y': 1, 'N': 0})
        else:
            raise RuntimeError(flag_name, values)


@FunctionTransformer
@copy_and_return_x
def sanitize_feature_names(x, y=None):
    x.columns = [re.sub(r'[^A-Za-z0-9_]+', '', str(col)) for col in x.columns.tolist()]


# 2. Build the pipeline
preprocessing_pipeline = Pipeline([
    ('custom_preprocessor', custom_preprocessor),
    ('occupation_pca', OccupationPCA()),
    ('first_name_sanitization', sanitize_feature_names),
    ('dummy_preprocessor', CategoricalEncoder(handle_unknown='ignore', sparse_output=False)),
], verbose=True)

# We try to see if the pipeline works as expected
X_train = train.copy()
y_train = X_train.pop('TARGET')
preprocessing_pipeline.fit(X_train, y_train)

[Pipeline]  (step 1 of 4) Processing custom_preprocessor, total=   1.1s
[Pipeline] .... (step 2 of 4) Processing occupation_pca, total=   1.2s
[Pipeline]  (step 3 of 4) Processing first_name_sanitization, total=   0.2s
[Pipeline]  (step 4 of 4) Processing dummy_preprocessor, total=   0.1s


,steps,"[('custom_preprocessor', ...), ('occupation_pca', ...), ...]"
,transform_input,None
,memory,None
,verbose,True
,func,<function cus...x78bb641c0720>
,inverse_func,None
,validate,False
,accept_sparse,False
,check_inverse,True
,feature_names_out,None
,kw_args,None


# Partie 4: Outils

In [12]:
# TODO: Danger of this, if values are negatives and test values are smaller than train values, offset won't be enough
# I don't want the BoxCox transformer to mess up data that it cannot fix.
# TODO: Review how I estimate if it's worth the transformation
epsilon = 1e-6


class ConditionalBoxCox(BaseEstimator, TransformerMixin):

    def __init__(self, p_value_threshold: float = 0.05):
        self.p_value_threshold = p_value_threshold
        self.columns_to_transform_ = []
        self.learned = {}

    def _as_numpy(self, X: pd.DataFrame | np.ndarray[tuple[int, ...], np.dtype[np.float64]]):
        x: np.ndarray
        if isinstance(X, pd.DataFrame):
            for col in X.columns:
                if X[col].isna().any():
                    raise NotImplementedError(
                        'I do not know yet how this would be handled,\n\t'
                        'for now, you need this to follow an inputer to have no null values')
            columns_to_drop = [i for i in X.columns if X[i].dtype == 'object']
            columns_to_keep = [i for i in X.columns if X[i].dtype in ('float64', 'int64')]
            if len(other_columns := set(X.columns) - set(columns_to_keep) - set(columns_to_drop)):
                raise NotImplementedError(other_columns)
            x = X.to_numpy()
        elif isinstance(X, np.ndarray):
            x = X
        else:
            raise NotImplementedError(type(X))
        return x

    # noinspection PyTypeHints
    def fit(self, X: pd.DataFrame | np.ndarray[tuple[int, ...], np.dtype[np.float64]], y=None):
        x = self._as_numpy(X.copy())

        for col, data in enumerate(x.T):
            if len(set(data)) < 10:  # Not worth the change
                continue
            data = data + (shift := epsilon - (0 if (data > 0).all() else data.min()))
            _, initial_p = stats.shapiro(sample(list(data), 100))
            transformed_data, ld = stats.boxcox(data)
            _, transformed_p = stats.shapiro(sample(list(transformed_data), 100))
            if transformed_p > initial_p and transformed_p > self.p_value_threshold:
                self.columns_to_transform_.append(col)
                self.learned[col] = ld, shift
        return self

    def transform(self, X):
        x = self._as_numpy(X.copy())
        for index in self.columns_to_transform_:
            ld, shift = self.learned[index]
            x[:, index] = stats.boxcox(
                x[:, index] + shift, lmbda=ld)
        return x


def new_pipeline(*steps):
    return Pipeline(deepcopy(preprocessing_pipeline.steps) + list(steps))

## Partie 5: Pipelines de modélisation

Nous allons maintenant définir nos modèles et lancer les expérimentations. Chaque modèle sera testé dans un `run` MLflow distinct.

Pour chaque `run`, nous allons logger :
1.  **Les paramètres** : Tous les hyperparamètres du pipeline.
2.  **Les métriques** : Les scores (AUC, coût métier, seuil) calculés par validation croisée (moyenne et écart-type).
3.  **Le modèle final** : Le pipeline complet, entraîné sur l'ensemble des données, prêt à être déployé.
4.  **(Optionnel) Des artefacts** : Comme des graphiques (ex: feature importance).

In [13]:
from mlflow import set_experiment, start_run, log_params, log_metric, log_param, models, sklearn

# TODO: `random_state`

# Définition des modèles de prétraitement
pipelines = [
    ('DummyClassifier', new_pipeline(('model', DummyClassifier()))),
    ('LogisticRegression', new_pipeline(
        ('imputer', SimpleImputer()),
        ('coxbox', ConditionalBoxCox()),
        ('scaler', StandardScaler()),
        ('model', LogisticRegression(class_weight='balanced')))),
    ('LightGBMClassifier', new_pipeline(('model', LGBMClassifier(class_weight='balanced'))))
]

# Définition de la stratégie de validation croisée
cv_strategy = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42)

X_train = train.copy()
y_train = X_train.pop('TARGET')
preprocessing_pipeline.fit_transform(X_train, y_train)
X_train = train.copy()
y_train = X_train.pop('TARGET')

display(Markdown(f"***{len(pipelines)}*** modèles sont prêts à être testés."))

# MLFlow
set_tracking_uri('http://127.65.12.247:50222')
set_experiment("P7_Credit_Scoring_Models")

# Boucle d'expérimentation
for model_name, pipeline in pipelines:
    print(f"--- Lancement du run pour le modèle : {model_name} ---")

    with start_run(run_name=model_name) as run:
        # 1. Log des paramètres
        # On log les paramètres pertinents de manière explicite pour plus de clarté
        log_params(pipeline.get_params()['model'].get_params())
        log_param("pipeline_steps", [step[0] for step in pipeline.steps])

        # 2. Entraînement et évaluation par validation croisée
        cv_results = cross_validate(pipeline, X, y, cv=cv_strategy, scoring=scoring, n_jobs=-1)

        # 3. Log des métriques (moyenne et écart-type pour chaque score)
        for score_name, values in cv_results.items():
            if 'test_' in score_name:
                metric_name = score_name.replace('test_', '')
                log_metric(f"{metric_name}_mean", values.mean())
                log_metric(f"{metric_name}_std", values.std())
            if 'time' in score_name:
                log_metric(f"{score_name}_mean", values.mean())
                log_metric(f"{score_name}_std", values.std())

        print(f"Coût métier moyen : {-cv_results['test_business_cost'].mean():.2f}")
        print(f"AUC moyen : {cv_results['test_roc_auc'].mean():.3f}")

        # 4. Entraînement du modèle final sur toutes les données
        pipeline.fit(X, y)

        # 5. Log du modèle avec sa signature
        # La signature aide MLflow à comprendre le format d'entrée/sortie
        sklearn.log_model(
            sk_model=pipeline,
            artifact_path="model",
            signature=models.infer_signature(X.head(), pipeline.predict_proba(X.head()))
        )

    print(f"--- Run pour {model_name} terminé et loggé. ---\n")

[Pipeline]  (step 1 of 4) Processing custom_preprocessor, total=   1.2s
[Pipeline] .... (step 2 of 4) Processing occupation_pca, total=   0.8s
[Pipeline]  (step 3 of 4) Processing first_name_sanitization, total=   0.2s
[Pipeline]  (step 4 of 4) Processing dummy_preprocessor, total=   0.8s


***3*** modèles sont prêts à être testés.

2025/08/23 11:40:55 INFO mlflow.tracking.fluent: Experiment with name 'P7_Credit_Scoring_Models' does not exist. Creating a new experiment.


--- Lancement du run pour le modèle : DummyClassifier ---


/home/nadir/Projects/p7.13.env/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:971: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/nadir/Projects/p7.13.env/lib/python3.13/site-packages/sklearn/metrics/_scorer.py", line 152, in __call__
    score = scorer._score(
        cached_call, estimator, *args, **routed_params.get(name).score
    )
  File "/home/nadir/Projects/p7.13.env/lib/python3.13/site-packages/sklearn/metrics/_scorer.py", line 408, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: business_cost_scorer_func() got an unexpected keyword argument 'needs_proba'

  warnings.warn(
/home/nadir/Projects/p7.13.env/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:971: UserWarning: Scoring failed. The score on 

...

/home/nadir/Projects/p7.13.env/lib/python3.13/site-packages/sklearn/impute/_base.py:637: UserWarning: Skipping features without any observed values: ['FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'
 'FLAG_PHONE' 'FLAG_EMAIL' 'FLAG_DOCUMENT_2' 'FLAG_DOCUMENT_3'
 'FLAG_DOCUMENT_4' 'FLAG_DOCUMENT_5' 'FLAG_DOCUMENT_6' 'FLAG_DOCUMENT_7'
 'FLAG_DOCUMENT_8' 'FLAG_DOCUMENT_9' 'FLAG_DOCUMENT_10' 'FLAG_DOCUMENT_11'
 'FLAG_DOCUMENT_12' 'FLAG_DOCUMENT_13' 'FLAG_DOCUMENT_14'
 'FLAG_DOCUMENT_15' 'FLAG_DOCUMENT_16' 'FLAG_DOCUMENT_17'
 'FLAG_DOCUMENT_18' 'FLAG_DOCUMENT_19' 'FLAG_DOCUMENT_20'
 'FLAG_DOCUMENT_21']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/home/nadir/Projects/p7.13.env/lib/python3.13/site-packages/sklearn/impute/_base.py:637: UserWarning: Skipping features without any observed values: ['FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'
 'FLAG_PHONE' 'FLAG_EMAIL' 'FLAG_DOCUMENT_2' 'FLAG_DOCUMENT_3'
 

🏃 View run LogisticRegression at: http://127.65.12.247:50222/#/experiments/886039402936229232/runs/4be9e6b07e8c4be680a38d403b95c282
🧪 View experiment at: http://127.65.12.247:50222/#/experiments/886039402936229232
--- Run pour LogisticRegression terminé et loggé. ---

--- Lancement du run pour le modèle : LightGBMClassifier ---


[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.


🏃 View run LightGBMClassifier at: http://127.65.12.247:50222/#/experiments/886039402936229232/runs/35a764cab7254a8bb9137b74e8339dc8
🧪 View experiment at: http://127.65.12.247:50222/#/experiments/886039402936229232


ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/nadir/Projects/p7.13.env/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nadir/Projects/p7.13.env/lib/python3.13/site-packages/sklearn/base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/nadir/Projects/p7.13.env/lib/python3.13/site-packages/sklearn/pipeline.py", line 663, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nad

### Partie 1.4.5: Comparison

In [ ]:
X_train = train.copy()
y_train = X_train.pop('TARGET')

scores = dict(
    (name, cross_val_score(pipeline, X_train, y_train, cv=RepeatedStratifiedKFold(
        n_splits=5, n_repeats=10, random_state=42))) for name, pipeline in pipelines)

clear_output(True)